In [1]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup
# regular expression
import re
# numerical analysis
import numpy as np

In [2]:
# get data
# ========

# link at which web data recides
link = 'https://www.worldometers.info/coronavirus/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# find the table
# ==============
# our target table is the last table in the page

# get the table head
# table head may contain the column names, titles, subtitles
thead = soup.find_all('thead')[-1]
# print(thead)

# get all the rows in table head
# it usually have only one row, which has the column names
head = thead.find_all('tr')
# print(head)

# get the table tbody
# it contains the contents
tbody = soup.find_all('tbody')[0]
# print(tbody)

# get all the rows in table body
# each row is each state's entry
body = tbody.find_all('tr')
# print(body)

In [4]:
# get the table contents
# ======================

# container for header rows / column title
head_rows = []
# container for table body / contents
body_rows = []

# loop through the head and append each row to head
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

# loop through the body and append each row to body
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [5]:
# save contents in a dataframe
# ============================
    
# skip last 3 rows, it contains unwanted info
# head_rows contains column title
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

# Drop 'S. No.' column
# df_bs.drop('S. No.', axis=1, inplace=True)

# there are 36 states+UT in India
df_bs.head(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,,\nNorth America\n,"4,643,235","+6,143","197,596",+350,"2,274,608","+5,256","2,171,031","23,663",,,,,,North America,\n,,
1,,\nSouth America\n,"3,308,065","+1,409","119,366",+67,"2,176,310",+322,"1,012,389","14,070",,,,,,South America,\n,,
2,,\nAsia\n,"3,465,482","+5,936","81,428",+97,"2,465,911","+7,001","918,143","19,546",,,,,,Asia,\n,,
3,,\nEurope\n,"2,694,711","+6,726","199,523",+180,"1,604,042","+9,754","891,146","5,331",,,,,,Europe,\n,,
4,,\nAfrica\n,"740,026",,"15,465",,"392,377",,"332,184","1,047",,,,,,Africa,\n,,


In [6]:
# drop unwanted rows
df_bs = df_bs.iloc[8:, :-3].reset_index(drop=True)

# drop unwanted columns
df_bs = df_bs.drop('#', axis=1)

# first few rows
df_bs.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent
0,USA,"3,961,556",+127,"143,835",+1,"1,850,224",+235,"1,967,497","16,616","11,965",434,"49,164,072","148,483","331,108,144",North America
1,Brazil,"2,121,645",,"80,251",,"1,409,202",,"632,192","8,318","9,978",377,"4,911,063","23,096","212,640,618",South America
2,India,"1,156,189","+1,272","28,099",,"724,702",,"403,388","8,944",837,20,"14,381,303","10,416","1,380,715,311",Asia
3,Russia,"783,328","+5,842","12,580",+153,"562,384","+8,782","208,364","2,300","5,368",86,"25,449,167","174,383","145,938,027",Europe
4,South Africa,"373,628",,"5,173",,"194,865",,"173,590",539,"6,296",87,"2,505,646","42,220","59,346,837",Africa


In [7]:
# rename columns
df_bs.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']

# rearrange and subselect columns
df_bs = df_bs[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

# first few rows
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"331,108,144","3,961,556",+127,"143,835",+1,"1,850,224",+235,"1,967,497","16,616","11,965",434,"49,164,072","148,483"
1,Brazil,South America,"212,640,618","2,121,645",,"80,251",,"1,409,202",,"632,192","8,318","9,978",377,"4,911,063","23,096"
2,India,Asia,"1,380,715,311","1,156,189","+1,272","28,099",,"724,702",,"403,388","8,944",837,20,"14,381,303","10,416"
3,Russia,Europe,"145,938,027","783,328","+5,842","12,580",+153,"562,384","+8,782","208,364","2,300","5,368",86,"25,449,167","174,383"
4,South Africa,Africa,"59,346,837","373,628",,"5,173",,"194,865",,"173,590",539,"6,296",87,"2,505,646","42,220"


In [8]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [9]:
# addding 'WHO Region' column
df_bs['WHO Region'] = df_bs['Country/Region'].map(who_region)

# countries with missing "WHO Region"
df_bs[df_bs['WHO Region'].isna()]['Country/Region'].unique()

array(['French Guiana', 'CAR', 'Mayotte', 'Diamond Princess', 'Réunion',
       'Channel Islands', 'Isle of Man', 'Martinique', 'Cayman Islands',
       'Guadeloupe', 'Faeroe Islands', 'Gibraltar', 'Brunei ', 'Aruba',
       'Turks and Caicos', 'Sint Maarten', 'French Polynesia',
       'St. Vincent Grenadines', 'Saint Martin', 'Macao', 'Curaçao',
       'New Caledonia', 'Falkland Islands', 'Montserrat',
       'Caribbean Netherlands'], dtype=object)

In [10]:
# fix data
for col in df_bs.columns[2:]:
    # replace comma with empty string
    df_bs[col] = df_bs[col].str.replace('[,+ ]', '', regex=True)
    # replace 'N/A' with empty string
    df_bs[col] = df_bs[col].str.replace('N/A', '', regex=False)

# replace empty strings with np.nan
df_bs = df_bs.replace('', np.nan)

# first few rows
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,331108144,3961556,127,143835,1,1850224,235,1967497,16616,11965,434,49164072,148483,Americas
1,Brazil,South America,212640618,2121645,NaN,80251,NaN,1409202,NaN,632192,8318,9978,377,4911063,23096,Americas
2,India,Asia,1380715311,1156189,1272,28099,NaN,724702,NaN,403388,8944,837,20,14381303,10416,South-EastAsia
3,Russia,Europe,145938027,783328,5842,12580,153,562384,8782,208364,2300,5368,86,25449167,174383,Europe
4,South Africa,Africa,59346837,373628,NaN,5173,NaN,194865,NaN,173590,539,6296,87,2505646,42220,Africa


In [11]:
# save as .csv file
df_bs.to_csv('worldometer_data.csv', index=False)